In [ ]:
#!unzip "/content/drive/My Drive/data/google-quest-challenge.zip" -d "/content/drive/My Drive/data/google-quest-challenge"

%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.text import *
from fastai.tabular import *
from fastai.callbacks import *
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#from google.colab import drive
#drive.mount('/content/drive')

path= Path("/home/jupyter/data/courtlistener")
#path= Path("/home/jupyter/data/courtlistener")

bs =64
df = pd.read_csv(path/"cl_scotus_opinions.csv")[0:1000]
print(df.shape)
print(df.head())
df = df.fillna('')
train, test = train_test_split(df, test_size=0.2)
#sub = pd.read_csv("/content/drive/My Drive/data/google-quest-challenge/sample_submission.csv")
train.head()
#preprocess
lm_df = train.append(test, ignore_index=True, sort=False)
np.random.seed(42)
lm_df = lm_df.iloc[np.random.permutation(len(train))]
cut = int(0.2 * len(lm_df)) +1
train_lm_df, valid_lm_df = lm_df[cut:], lm_df[:cut]

data_lm = TextLMDataBunch.from_df(path, train_lm_df, valid_lm_df, text_cols=["citation", "resource_type","jurisdiction","category","judge","summary","text","float_dummy1","float_dummy2"],
                                  mark_fields=True,
                                  bs=bs)
data_lm.show_batch()
data_lm.save('data_lm_export.pkl')
labels = train.columns[(train.columns.str.startswith("float_dummy")) | 
                           (train.columns.str.startswith("float_dummy"))].to_list()
labels = list(filter(lambda x: x not in ['text',
                                         'judge',
                                         'resource_type',
                                         'category'], labels))
print(labels)

#assert len(labels) == 30
data_clas = TextClasDataBunch.from_df(path, train, test,
                                       vocab=data_lm.train_ds.vocab, bs=bs,
                                       text_cols=["citation", "case_name"],
                                       mark_fields=True,
                                       label_cols=labels)
data_clas.show_batch(reverse=True)
data_clas.save('data_clas.pkl')

#fine tune LM
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)

learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
#learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5,
                               #metrics=[accuracy, Perplexity()],
                               #callback_fns=[partial(EarlyStoppingCallback, monitor="perplexity", mode="min", patience=3),
                                             #partial(SaveModelCallback, monitor="perplexity", mode="min"), name='bestmodel'])
learn.save('bestmodel.pth')
learn = learn.to_fp16()
learn.lr_find()
learn.recorder.plot(skip_end=5)
learn.fit_one_cycle(1, 5e-2, moms=(0.8, 0.7))
learn.save('fit_head')
learn.load('fit_head');
#to complete fine-tuning, unfreeze and launch a new training
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3, moms=(0.8, 0.7)) #todo default 10
learn.save('fine_tuned')
learn.load('fine_tuned')                                                         
learn.export(file=Path("/home/jupyter/data/courtlistener/courtlistener_text_generator_LSTM_export_v2.pkl"))
#print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))
print(learn.predict("According to constitution, the federal rules, and stare decisis", n_words=100))
learn.save_encoder('fine_tuned_enc')



In [1]:
TEXT="Under state law in the state of"
N_WORDS = 199
N_SENTENCES=2
learn = load_learner(Path('/content/drive/My Drive/data/courtlistener'),'courtlistener_text_generator_LSTM_export_v2.pkl')
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))


NameError: ignored

In [ ]:
#Classifier
#TODO
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.text import *
from google.colab import drive
drive.mount('/content/drive')
data_lm = load_data(path,'data_lm.pkl', bs=bs) 
path= Path('/content/drive/My Drive/data/judges')
bs=16
data_clas = (TextList.from_folder(path, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             .label_from_folder(classes=['alito','black','brandeis', 'breyer','clifford', 'ginsburg', 'harlan','holmes', 'kennedy', 'marshall', 'mathews', 'oconnor', 'robers','rutledge', 'scalia', 'sotomayor', 'souter', 'taney', 'thomas', 'todd', 'warren'])
             #label them all with their folders
             .databunch(bs=bs))

data_clas.save('data_clas.pkl')



data_clas.save('data_clas.pkl')
data_clas = load_data(path, 'data_clas.pkl', bs=bs)
#data_clas.show_batch()

#creatae a model to classify those reviews and load the encoder we saved before
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('fine_tuned_enc')
learn.lr_find()
learn.recorder.plot()
learn.fit_one_cycle(1, 2e-2, moms=(.8, .7))

learn.save('first')
learn.load('first');
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2), moms=(0.8,0.7))
learn.save('second')
learn.load('second')
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3), moms=(0.8,0.7))
learn.save('third')
learn.load('third')

learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3), moms=(0.8,0.7))
learn.export(file=Path("/content/drive/My Drive/data/courtlistener_text_classifier_LSTM_export_v2.pkl"))
learn.predict("The holding in the case at hand")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: ignored